In [2]:
import psycopg2

In [3]:
try:
    conn = psycopg2.connect("dbname=udacity user=udacity password=udacity")
except Exception as e:
    print(e)

try:
    cur = conn.cursor()
except Exception as e:
    print(e)

conn.set_session(autocommit=True)

# Create tables and insert data

## Create fact table

In [20]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS customer_transactions \
                (customer_id int, store_id int, spent numeric)")
except Exception as e:
    print(e)

## Insert data in fact table

In [10]:
try:
    cur.execute("INSERT INTO customer_transactions \
                (customer_id, store_id, spent) \
                VALUES (%s, %s, %s)", \
                (1, 1, 20.50))
except Exception as e:
    print(e)

try:
    cur.execute("INSERT INTO customer_transactions \
                (customer_id, store_id, spent) \
                VALUES (%s, %s, %s)", \
                (2, 1, 35.21))
except Exception as e:
    print(e)

## Create dimension tables

In [21]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS customer \
                (customer_id int, name varchar, rewards boolean)")
except Exception as e:
    print(e)

try:
    cur.execute("CREATE TABLE IF NOT EXISTS store \
                (store_id int, state varchar)")
except Exception as e:
    print(e)

try:
    cur.execute("CREATE TABLE IF NOT EXISTS items_purchased \
                (customer_id int, item_number int, item_name varchar)")
except Exception as e:
    print(e)

## Insert data in Dimension tables

In [5]:
# Customer Table
try:
    cur.execute("INSERT INTO customer \
                (customer_id, name, rewards) \
                VALUES (%s, %s, %s)", \
                (1, "Amanda", True))
except Exception as e:
    print(e)

try:
    cur.execute("INSERT INTO customer \
                (customer_id, name, rewards) \
                VALUES (%s, %s, %s)", \
                (2, "Toby", False))
except Exception as e:
    print(e)

# Store table
try:
    cur.execute("INSERT INTO store \
                (store_id, state) \
                VALUES (%s, %s)", \
                (1, "CA"))
except Exception as e:
    print(e)

try:
    cur.execute("INSERT INTO store \
                (store_id, state) \
                VALUES (%s, %s)", \
                (2, "WA"))
except Exception as e:
    print(e)

# Items Purchased table

try:
    cur.execute("INSERT INTO items_purchased \
                (customer_id, item_number, item_name) \
                VALUES (%s, %s, %s)", \
                (1, 1, "Rubber Soul"))
except Exception as e:
    print(e)

try:
    cur.execute("INSERT INTO items_purchased \
                (customer_id, item_number, item_name) \
                VALUES (%s, %s, %s)", \
                (2, 3, "Let It Be"))
except Exception as e:
    print(e)


# SQL queries

## Query 1: find all the customers that spent more than 30 dollars, who are they, what did they buy and if they are a reward member

In [4]:
try:
    cur.execute("SELECT name, item_name, rewards FROM customer_transactions \
                JOIN customer ON customer_transactions.customer_id = customer.customer_id \
                JOIN items_purchased ON customer_transactions.customer_id = items_purchased.customer_id \
                WHERE spent > 30")
except Exception as e:
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Toby', 'Let It Be', False)


## Query 2: How much did Store 1 sell

In [5]:
try:
    cur.execute("SELECT store.store_id, SUM(spent) FROM customer_transactions \
                JOIN store ON customer_transactions.store_id = store.store_id \
                GROUP BY store.store_id")
except Exception as e:
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, Decimal('55.71'))


# Drop Tables

In [22]:
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")

query = "DROP TABLE "
tables = cur.fetchall()

for table in tables:
       try:
              cur.execute(query + table[0])
              print("Table " + table [0] + " deleted")
       except Exception as e:
              print(e)

Table customer_transactionsdeleted
Table customerdeleted
Table storedeleted
Table items_purchaseddeleted


# Close cursor and connection

In [ ]:
cur.close()
conn.close()